In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
import sys
sys.path.append('..')

import autorobot as ar
from autorobot.constants import (
    RProjType,
)

In [4]:
rb = ar.initialize()

In [5]:
from autorobot.synonyms import synonyms
synonyms.update({'seashell': ar.RProjType.SHELL})

print(ar.app.synonyms)

{'BUILDING': <RProjType.BUILDING: 19>, 'FRAME_2D': <RProjType.FRAME_2D: 1>, 'FRAME_3D': <RProjType.FRAME_3D: 4>, 'SHELL': <RProjType.SHELL: 7>, 'TRUSS_2D': <RProjType.TRUSS_2D: 2>, 'TRUSS_3D': <RProjType.TRUSS_3D: 5>, 'PERM': <RCaseNature.PERM: 0>, 'IMPOSED': <RCaseNature.IMPOSED: 1>, 'WIND': <RCaseNature.WIND: 2>, 'SNOW': <RCaseNature.SNOW: 3>, 'ACC': <RCaseNature.ACC: 5>, 'SLS': <RCombType.SLS: 1>, 'ULS': <RCombType.ULS: 0>, 'LINEAR': <RAnalysisType.LINEAR: 1>, 'NON_LIN': <RAnalysisType.NON_LIN: 2>, 'COMB_LINEAR': <RAnalysisType.COMB_LINEAR: 0>, 'COMB_NON_LIN': <RAnalysisType.COMB_NON_LIN: -1>, <RCaseNature.PERM: 0>: 0, <RCaseNature.IMPOSED: 1>: 1, <RCaseNature.WIND: 2>: 2, <RCaseNature.SNOW: 3>: 3, <RCaseNature.ACC: 5>: 5, <RProjType.BUILDING: 19>: 19, <RProjType.FRAME_3D: 4>: 4, <RProjType.SHELL: 7>: 7, <RAnalysisType.COMB_NON_LIN: -1>: -1, 'seashell': <RProjType.SHELL: 7>}


In [6]:
rb.new('seashell')

In [7]:
for attr in dir(ar.RobotOM.IRobotBarSectionShapeType):
    if attr.startswith('I_BSST'):
        print(f'{attr}: {getattr(ar.RobotOM.IRobotBarSectionShapeType, attr)}')

I_BSST_BOX: 91
I_BSST_CAE: 1
I_BSST_CAEP: 2
I_BSST_CAI: 3
I_BSST_CAIP: 4
I_BSST_CCL: 45
I_BSST_CIRC_FILLED: 44
I_BSST_COLD_ANGL: 1006
I_BSST_COLD_CE1: 1005
I_BSST_COLD_C_PLUS: 1010
I_BSST_COLD_L_LIPS: 1014
I_BSST_COLD_OMEGA: 1007
I_BSST_COLD_RIVE1: 1009
I_BSST_COLD_SIGMA: 1012
I_BSST_COLD_SIGMA1: 1001
I_BSST_COLD_SIGMA2: 1002
I_BSST_COLD_SIGMA_SL: 1011
I_BSST_COLD_SO1: 1008
I_BSST_COLD_U: 1004
I_BSST_COLD_Z: 1013
I_BSST_COLD_ZED1: 1003
I_BSST_COLD_Z_ROT: 1015
I_BSST_COMP_2C_BACK: 1102
I_BSST_COMP_2C_BACK_WELD: 1202
I_BSST_COMP_2C_FACE: 1101
I_BSST_COMP_2C_FACE_WELD: 1201
I_BSST_COMP_2I: 1103
I_BSST_COMP_2I_WELD: 1203
I_BSST_COMP_2LI: 1105
I_BSST_COMP_2LI_WELD: 1205
I_BSST_COMP_2L_CROSS: 1110
I_BSST_COMP_2L_CROSS_WELD: 1210
I_BSST_COMP_2L_FACE_LONG: 1112
I_BSST_COMP_2L_FACE_LONG_WELD: 1212
I_BSST_COMP_2L_FACE_SHORT: 1111
I_BSST_COMP_2L_FACE_SHORT_WELD: 1211
I_BSST_COMP_2L_LONG: 1109
I_BSST_COMP_2L_LONG_WELD: 1209
I_BSST_COMP_2L_SHORT: 1108
I_BSST_COMP_2L_SHORT_WELD: 1208
I_BSST_COMP_4L_

In [8]:
rb.sections.load('UB 305x165x40')
section = rb.sections.get('UB 305x165x40')

In [9]:
section.data.Type

0

In [10]:

rb.nodes.create(0,0,0)
rb.nodes.create(1,1,1)

In [11]:
rb.supports.create('New Uy', '111000', node=1, orient_node=2)

In [12]:
uy = rb.supports.get('New Uy')
print(uy.data.Alpha, uy.data.Beta, uy.data.Gamma)

0.7853981633974483 0.9553166181245092 0.0
